# Necessary Libraries

In [1]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install nltk

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

# Reading in the corpus
For our example,we will be using the Wikipedia page for chatbots as our corpus. Copy the contents from the page and place it in a text file named ‘chatbot.txt’. However, you can use any corpus of your choice.

In [5]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase

## Tokenization

In [7]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words

## Pre-Processing

In [8]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Keyword Matching

In [9]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):

    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Generating response

In [10]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [11]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hello
ROBO: hi there
do you know astronomy?
ROBO: I am sorry! I don't understand you
what is a chatbot
ROBO: design
the chatbot design is the process that defines the interaction between the user and the chatbot.the chatbot designer will define the chatbot personality, the questions that will be asked to the users, and the overall interaction.it can be viewed as a subset of the conversational design.
who are you?
ROBO: I am sorry! I don't understand you
can you talk?
ROBO: I am sorry! I don't understand you
bye
ROBO: Bye! take care..


# Improving the Chatbot using transformers

In [12]:
!pip install transformers
!pip install torch
!pip install flask

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [17]:
!pip install pyngrok

# Code for Improved Response Generation

In [14]:
from transformers import pipeline

# Load a pre-trained model and tokenizer from Hugging Face
chatbot = pipeline("text2text-generation", model="microsoft/DialoGPT-medium")

def get_response(user_input):
    response = chatbot(user_input)
    return response.generated_responses[0]


pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

The model 'GPT2LMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


# Basic Context Management
 We can use a simple dictionary to manage context within a single conversation session.

In [15]:
context = {}

def manage_context(user_id, user_input):
    if user_id not in context:
        context[user_id] = []
    context[user_id].append(user_input)
    return context[user_id]


# Basic Web Interface using Flask
We'll create a simple Flask web application to interact with the chatbot.

In [39]:
from flask import Flask, request, jsonify, render_template_string
from transformers import BlenderbotSmallTokenizer, BlenderbotSmallForConditionalGeneration
from pyngrok import ngrok

app = Flask(__name__)

# Load a pre-trained BlenderbotSmall model and tokenizer from Hugging Face
tokenizer = BlenderbotSmallTokenizer.from_pretrained("facebook/blenderbot_small-90M")
model = BlenderbotSmallForConditionalGeneration.from_pretrained("facebook/blenderbot_small-90M")

context = {}

def manage_context(user_id, user_input):
    if user_id not in context:
        context[user_id] = []
    context[user_id].append(user_input)
    return context[user_id]

def get_response(user_id, user_input):
    user_context = manage_context(user_id, user_input)
    inputs = tokenizer(" ".join(user_context), return_tensors="pt")
    reply_ids = model.generate(**inputs)
    response = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
    return response

@app.route("/")
def home():
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Chatbot</title>
        <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    </head>
    <body>
        <h1>Hello World</h1>
        <div id="chat-box">
            <div id="chat-log"></div>
            <input type="text" id="user-input" placeholder="Type your message...">
            <button onclick="sendMessage()">Send</button>
        </div>

        <script>
            function sendMessage() {
                var user_id = Math.random().toString(36).substr(2, 9); // generate random user id
                var user_input = $("#user-input").val();
                $.post("/get", {msg: user_input, user_id: user_id}, function(data) {
                    $("#chat-log").append("<p>You: " + user_input + "</p>");
                    $("#chat-log").append("<p>Bot: " + data.response + "</p>");
                    $("#user-input").val("");
                });
            }
        </script>
    </body>
    </html>
    """)

@app.route("/get", methods=["POST"])
def get_bot_response():
    user_input = request.form["msg"]
    user_id = request.form["user_id"]
    response = get_response(user_id, user_input)
    return jsonify({"response": response})

# Run the app
!nohup flask run --host=0.0.0.0 --port=5000 &


nohup: appending output to 'nohup.out'


In [41]:
# Import and configure ngrok
from pyngrok import ngrok

# Run the Flask app in the background
from threading import Thread

def run_app():
    app.run(host="0.0.0.0", port=5000)

thread = Thread(target=run_app)
thread.start()
!ngrok authtoken 2jjsfF9yK3J4iGA1dSyviGmEcrc_5SB4thdANHDjWwgqAW2hw

# Now try to connect again
public_url = ngrok.connect(addr='5000')
print(f"Public URL: {public_url}")


 * Serving Flask app '__main__'
 * Debug mode: off
Public URL: NgrokTunnel: "https://86a3-35-223-12-215.ngrok-free.app" -> "http://localhost:5000"


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
